In [6]:
#Download example dataset(835 MB)
import gdown
url = "https://drive.google.com/u/1/uc?id=16-jgsWDYkdZdRPxnXMlRUdfyS3ihj4g_&export=download"
output = "small_ngvla_sim.zarr.zip"
gdown.download(url, output)

Downloading...
From: https://drive.google.com/u/1/uc?id=16-jgsWDYkdZdRPxnXMlRUdfyS3ihj4g_&export=download
To: /Users/jsteeb/Dropbox/viper/dev/small_ngvla_sim.zarr.zip
100%|██████████████████████████████████████████████████████████████████| 835M/835M [00:07<00:00, 113MB/s]


'small_ngvla_sim.zarr.zip'

In [7]:
import shutil
shutil.unpack_archive(filename="small_ngvla_sim.zarr.zip", extract_dir=".")

In [3]:
from viper.dio._utils._io import _load_mxds
mxds_name = 'small_ngvla_sim.zarr'
sel_parms = {'xds_id':[0,1,2]}
mxds = _load_mxds(mxds_name,sel_parms)

FileNotFoundError: No such file or directory: '/Users/jsteeb/Library/CloudStorage/Dropbox/viper/examples/small_ngvla_sim.zarr/xds0'

In [4]:
mxds[0] # 1, 2

NameError: name 'mxds' is not defined

In [3]:
#Create Graph
parallel_coords = mxds[0] #using first xds as the parallel_coords
parallel_dims = ['chan','time'] #['pol','time','baseline']

parallel_coords

<xarray.Dataset>
Dimensions:             (baseline: 14878, baseline_ids_names: 4, time: 100,
                         chan: 7, pol: 2, uvw: 3)
Coordinates:
  * baseline            (baseline) int64 0 1 2 3 4 ... 14874 14875 14876 14877
  * baseline_ids_names  (baseline_ids_names) <U8 'ANTENNA1' ... 'FEED2'
  * chan                (chan) float64 1.73e+10 2.016e+10 ... 3.444e+10
  * pol                 (pol) int64 9 12
  * time                (time) float64 5.145e+09 5.145e+09 ... 5.145e+09
Dimensions without coordinates: uvw
Data variables:
    BASELINE_IDS        (baseline, baseline_ids_names) int64 dask.array<chunksize=(14878, 4), meta=np.ndarray>
    DATA                (time, baseline, chan, pol) complex128 dask.array<chunksize=(25, 14878, 3, 2), meta=np.ndarray>
    FIELD_ID            (time, baseline) int64 dask.array<chunksize=(25, 14878), meta=np.ndarray>
    FLAG                (time, baseline, chan, pol) bool dask.array<chunksize=(25, 14878, 3, 2), meta=np.ndarray>
    SCAN_IDS            (time) float64 dask.array<chunksize=(25,), meta=np.ndarray>
    TIME_CENTROID       (time, baseline) float64 dask.array<chunksize=(25, 14878), meta=np.ndarray>
    UVW                 (time, baseline, uvw) float64 dask.array<chunksize=(25, 14878, 3), meta=np.ndarray>
    WEIGHT              (time, baseline, chan, pol) float64 dask.array<chunksize=(25, 14878, 3, 2), meta=np.ndarray>

In [4]:
def cube_imaging_chunk(input_parms):
    from viper._utils._io import _open_no_dask_zarr
    import os
    mxds_name = input_parms['mxds_name']
    
    mxds = {}
    for xds_id in input_parms['data_sel']:
        mxds[xds_id] =_open_no_dask_zarr(os.path.join(mxds_name,'xds'+str(xds_id)),input_parms['data_sel'][xds_id])
        print(mxds,'\n ********')
    
    return "Hallo from chunk" + str(input_parms["chunk_id"])

In [5]:
from viper._utils._graph_tools import _build_perfectly_parallel_graph
graph = _build_perfectly_parallel_graph('small_ngvla_sim.zarr',sel_parms,parallel_coords=parallel_coords,parallel_dims=parallel_dims,func_chunk=cube_imaging_chunk)
graph = _build_gather_graph(graph)

[Delayed('cube_imaging_chunk-90de4c1c-5f90-4912-82dd-622b19ebfb76'),
 Delayed('cube_imaging_chunk-868f91e8-1480-4847-8351-422ddcaee0d2'),
 Delayed('cube_imaging_chunk-89830ec6-812e-48e1-8a56-bc2cf7e5b3d8'),
 Delayed('cube_imaging_chunk-8e1dc0b5-2660-41b0-b34c-b2afab24537c'),
 Delayed('cube_imaging_chunk-205c2fb3-ff18-4bda-8695-52956272c3ad'),
 Delayed('cube_imaging_chunk-ddf04414-2f0b-4623-b339-2544bb22c82d'),
 Delayed('cube_imaging_chunk-8f83669f-0058-4de6-9a92-313ffe219588'),
 Delayed('cube_imaging_chunk-6fdd849c-f88d-4022-9401-5f32584a48a1'),
 Delayed('cube_imaging_chunk-cea6120e-4a53-465b-bf63-c739e8fa9efe'),
 Delayed('cube_imaging_chunk-779ed266-04b1-4b1c-971e-c750283fbe0d'),
 Delayed('cube_imaging_chunk-2ceba336-f8b7-4e30-b97b-91944efa4c8c'),
 Delayed('cube_imaging_chunk-147952f9-ddec-4b91-8100-d9fa57646ddf')]

In [7]:
import dask
dask.visualize(graph,filename='perfectly_parallel_graph.png')

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [27]:
import dask
result = dask.compute(graph)

{0: <xarray.Dataset>
Dimensions:             (baseline: 14878, baseline_ids_names: 4, time: 25,
                         chan: 3, pol: 2, uvw: 3)
Coordinates:
  * baseline            (baseline) int64 0 1 2 3 4 ... 14874 14875 14876 14877
  * baseline_ids_names  (baseline_ids_names) <U8 'ANTENNA1' ... 'FEED2'
  * chan                (chan) float64 2.587e+10 2.873e+10 3.159e+10
  * pol                 (pol) int64 9 12
  * time                (time) float64 5.145e+09 5.145e+09 ... 5.145e+09
Dimensions without coordinates: uvw
Data variables:
    BASELINE_IDS        (baseline, baseline_ids_names) int64 0 1 0 0 ... 172 0 0
    DATA                (time, baseline, chan, pol) complex128 (4.99837166899...
    FIELD_ID            (time, baseline) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    FLAG                (time, baseline, chan, pol) int8 0 0 0 0 0 ... 0 0 0 0 0
    SCAN_IDS            (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TIME_CENTROID       (time, baseline) float64 5.1

In [28]:
result

(['Hallo from chunk(0, 0)',
  'Hallo from chunk(0, 1)',
  'Hallo from chunk(0, 2)',
  'Hallo from chunk(0, 3)',
  'Hallo from chunk(1, 0)',
  'Hallo from chunk(1, 1)',
  'Hallo from chunk(1, 2)',
  'Hallo from chunk(1, 3)',
  'Hallo from chunk(2, 0)',
  'Hallo from chunk(2, 1)',
  'Hallo from chunk(2, 2)',
  'Hallo from chunk(2, 3)'],)